In [3]:
import glob, os
import numpy as np
import pysptk
import librosa
import time
import pandas as pd
import sys
from scipy.io import wavfile

# Sound stimuli preparation 
Treadmill experiment &copy; Quentin Decultot, JJ Aucouturier, 2021

In [2]:
sound_dir = "../experiment/sounds/"

## Sound file conversion

Only select mf files, and convert from aiff to wav

In [3]:
# convert aiff to wav files
for input_file in glob.glob(sound_dir+"piano/Piano.mf.*.aiff"):
    output_file = os.path.splitext(input_file)[0] +'.wav'
    print(os.path.basename(output_file)+',', end='')
    convert_cmd = "ffmpeg -i " + input_file + " -acodec pcm_s16le -ar 44100 -ac 1 " + output_file
    os.system(convert_cmd)

Piano.mf.A1.wav,Piano.mf.A2.wav,Piano.mf.A3.wav,Piano.mf.A4.wav,Piano.mf.A5.wav,Piano.mf.A6.wav,Piano.mf.A7.wav,Piano.mf.Ab1.wav,Piano.mf.Ab2.wav,Piano.mf.Ab3.wav,Piano.mf.Ab4.wav,Piano.mf.Ab5.wav,Piano.mf.Ab6.wav,Piano.mf.Ab7.wav,Piano.mf.B0.wav,Piano.mf.B1.wav,Piano.mf.B2.wav,Piano.mf.B3.wav,Piano.mf.B4.wav,Piano.mf.B5.wav,Piano.mf.B6.wav,Piano.mf.B7.wav,Piano.mf.Bb1.wav,Piano.mf.Bb2.wav,Piano.mf.Bb3.wav,Piano.mf.Bb4.wav,Piano.mf.Bb5.wav,Piano.mf.Bb6.wav,Piano.mf.Bb7.wav,Piano.mf.C1.wav,Piano.mf.C2.wav,Piano.mf.C3.wav,Piano.mf.C4.wav,Piano.mf.C5.wav,Piano.mf.C6.wav,Piano.mf.C7.wav,Piano.mf.C8.wav,Piano.mf.D1.wav,Piano.mf.D2.wav,Piano.mf.D3.wav,Piano.mf.D4.wav,Piano.mf.D5.wav,Piano.mf.D6.wav,Piano.mf.D7.wav,Piano.mf.Db1.wav,Piano.mf.Db2.wav,Piano.mf.Db3.wav,Piano.mf.Db4.wav,Piano.mf.Db5.wav,Piano.mf.Db6.wav,Piano.mf.Db7.wav,Piano.mf.E1.wav,Piano.mf.E2.wav,Piano.mf.E3.wav,Piano.mf.E4.wav,Piano.mf.E5.wav,Piano.mf.E6.wav,Piano.mf.E7.wav,Piano.mf.Eb1.wav,Piano.mf.Eb2.wav,Piano.mf.Eb3.wav,

## Trim silence and normalize

Cut leading silence

In [4]:
duration = 1.5
fade_duration = 0.05

for input_file in glob.glob(sound_dir+"piano/*.wav"):
    
    output_file = sound_dir+"/original/"+os.path.splitext(os.path.basename(input_file))[0] +'.t.wav'
    trim_cmd = "ffmpeg -i " + input_file + (" -af loudnorm,"
                                            "aformat=s16,"
                                            "silenceremove=start_periods=1:start_silence=0.05:start_threshold=-30dB,"
                                            "afade=out:st=%f:d=%f,"
                                            "afade=in:st=0:d=%f -ar 44100 "
                                            "-to %f ")%(duration-fade_duration,
                                                        fade_duration, 
                                                        fade_duration, 
                                                        duration) + output_file
    os.system(trim_cmd)
    print(os.path.basename(output_file)+',', end='')


Piano.mf.A1.t.wav,Piano.mf.A2.t.wav,Piano.mf.A3.t.wav,Piano.mf.A4.t.wav,Piano.mf.A5.t.wav,Piano.mf.A6.t.wav,Piano.mf.A7.t.wav,Piano.mf.Ab1.t.wav,Piano.mf.Ab2.t.wav,Piano.mf.Ab3.t.wav,Piano.mf.Ab4.t.wav,Piano.mf.Ab5.t.wav,Piano.mf.Ab6.t.wav,Piano.mf.Ab7.t.wav,Piano.mf.B0.t.wav,Piano.mf.B1.t.wav,Piano.mf.B2.t.wav,Piano.mf.B3.t.wav,Piano.mf.B4.t.wav,Piano.mf.B5.t.wav,Piano.mf.B6.t.wav,Piano.mf.B7.t.wav,Piano.mf.Bb1.t.wav,Piano.mf.Bb2.t.wav,Piano.mf.Bb3.t.wav,Piano.mf.Bb4.t.wav,Piano.mf.Bb5.t.wav,Piano.mf.Bb6.t.wav,Piano.mf.Bb7.t.wav,Piano.mf.C1.t.wav,Piano.mf.C2.t.wav,Piano.mf.C3.t.wav,Piano.mf.C4.t.wav,Piano.mf.C5.t.wav,Piano.mf.C6.t.wav,Piano.mf.C7.t.wav,Piano.mf.C8.t.wav,Piano.mf.D1.t.wav,Piano.mf.D2.t.wav,Piano.mf.D3.t.wav,Piano.mf.D4.t.wav,Piano.mf.D5.t.wav,Piano.mf.D6.t.wav,Piano.mf.D7.t.wav,Piano.mf.Db1.t.wav,Piano.mf.Db2.t.wav,Piano.mf.Db3.t.wav,Piano.mf.Db4.t.wav,Piano.mf.Db5.t.wav,Piano.mf.Db6.t.wav,Piano.mf.Db7.t.wav,Piano.mf.E1.t.wav,Piano.mf.E2.t.wav,Piano.mf.E3.t.wav,Piano.m

## Compute pitch and list in a file

In [5]:
PITCH_WIN = .01
PITCH_BOUNDS = [20,4200] # minimum maximum freq on piano

def compute_swipe_pitch(sound_file):
    """Computes a time series of pitch values using the SWIPE algorithm
    Pitch is computed on successive windows (PITCH_WIN, in sec.), 
    and returns an array of pitch values (in Hz) and array of window time positions (in ms).
    PITCH_BOUNDS = [min_f0,max_f0] in which the pitch should be searched.
    Pitch on windows where pitch is undefined is set as np.nan
    """
    x, sr = librosa.load(sound_file) #sr=sampling_rate
    #sr, x = wavfile.read(sound_file)
    hop_size = np.floor(sr * PITCH_WIN)
    min_f0, max_f0 = PITCH_BOUNDS
    pitch = np.array(pysptk.swipe(x.astype(np.float64),
                         fs=sr,
                         hopsize=hop_size,
                         min=min_f0,
                         max=max_f0,
                         otype=1))
    
    # pitch on windows where pitch is undefined (ex. non-voiced) is returned as 0
    # replace these values by np.nan
    pitch[np.where(pitch == 0)] = np.nan
    
    # times at which pitch is computed 
    times = 1000*np.arange(len(pitch))*hop_size/sr
    
    return pitch, times


In [10]:
files = []
dynamics = [] 
pitches = []
notes = []
octaves = []

for input_file in glob.glob(sound_dir+"/original/*.t.wav"):
    
    base_name = os.path.basename(input_file)
    files.append("original/"+base_name)
    
    # parse note & dynamics from file name
    dynamics.append(base_name.split('.')[1])
    notes.append(base_name.split('.')[2])
    octaves.append(base_name.split('.')[2][-1])
    
    # compute pitch
    print("pitch %s: "%(base_name), end='')
    swipe_pitches, times = compute_swipe_pitch(input_file)
    swipe_pitches = swipe_pitches[~np.isnan(swipe_pitches)]
    pitch = np.median(swipe_pitches) # note: median instead of mean, to focus on the tonal parts
    print("%.1f Hz, "%pitch,end='')
    pitches.append(float("%.1f"%pitch))
                   
# list all sounds in a dataframe        
sounds = pd.DataFrame({"file":files,
                       'octave':octaves,
                      "note":notes,
                      "dynamics":dynamics,
                      "pitch":pitches})   
    
    


pitch Piano.mf.A1.t.wav: 55.4 Hz, pitch Piano.mf.A2.t.wav: 110.4 Hz, pitch Piano.mf.A3.t.wav: 221.0 Hz, pitch Piano.mf.A4.t.wav: 442.9 Hz, pitch Piano.mf.A5.t.wav: 895.4 Hz, pitch Piano.mf.A6.t.wav: 1818.4 Hz, pitch Piano.mf.A7.t.wav: 1699.4 Hz, pitch Piano.mf.Ab1.t.wav: 52.1 Hz, pitch Piano.mf.Ab2.t.wav: 104.2 Hz, pitch Piano.mf.Ab3.t.wav: 209.2 Hz, pitch Piano.mf.Ab4.t.wav: 419.1 Hz, pitch Piano.mf.Ab5.t.wav: 851.2 Hz, pitch Piano.mf.Ab6.t.wav: 1725.7 Hz, pitch Piano.mf.Ab7.t.wav: 2885.3 Hz, pitch Piano.mf.B0.t.wav: 31.0 Hz, pitch Piano.mf.B1.t.wav: 62.0 Hz, pitch Piano.mf.B2.t.wav: 124.0 Hz, pitch Piano.mf.B3.t.wav: 248.5 Hz, pitch Piano.mf.B4.t.wav: 501.6 Hz, pitch Piano.mf.B5.t.wav: 1016.0 Hz, pitch Piano.mf.B6.t.wav: 2087.7 Hz, pitch Piano.mf.B7.t.wav: 1383.3 Hz, pitch Piano.mf.Bb1.t.wav: 58.4 Hz, pitch Piano.mf.Bb2.t.wav: 117.1 Hz, pitch Piano.mf.Bb3.t.wav: 235.0 Hz, pitch Piano.mf.Bb4.t.wav: 470.5 Hz, pitch Piano.mf.Bb5.t.wav: 948.6 Hz, pitch Piano.mf.Bb6.t.wav: 1952.8 Hz, pitc

In [11]:
sounds.to_csv('../experiment/sounds/list.csv')

## Pitch shift

Pitch shift each file to [-50,+50] cents, to expand the dataset. Uses a proprietary method in pyvoimooo. 

In [12]:
# Load Voimooo python wrapper
sys.path.insert(0, os.path.abspath('../../../pyvoimooo/'))
import pyvoimooo as vmo
vmo.__version__

'0.17.6'

In [13]:
files = []
dynamics = [] 
pitches = []
notes = []
shifts = []
octaves = []

sounds= pd.read_csv('../experiment/sounds/list.csv', index_col=0)

for index, row in sounds.iterrows():
        
    # read base sound
    sound_file = sound_dir+"/"+row['file']
    
    #sr, x = wavfile.read(sound_file)
    x, sr = librosa.load(sound_file) #sr=sampling_rate
        
    # generate pitch shifted versions by [10:90] cents
    for cent in np.arange(-50,+50,10):
        
        pitch_scale_factor = np.power(2,cent/1200) # pitch factor 2^(n/1200)
        transformed = vmo.pitch_scaling_doubledelay(np.array(x), sr, sps= pitch_scale_factor)
        
        output_file = os.path.splitext(os.path.basename(row['file']))[0] + ".shift%+d.wav"%cent
        
        print("scale by %.2f: %s "%(pitch_scale_factor, output_file), end ='')
        wavfile.write(sound_dir + "/transformed/temp/" + output_file, sr, transformed)
        
        files.append('transformed/'+output_file)
        dynamics.append(row['dynamics'])
        notes.append(row['note'])
        octaves.append(row['octave'])
        pitches.append(row['pitch']*pitch_scale_factor)
        shifts.append(cent)
        

# save in a file        
sounds = pd.read_csv('../experiment/sounds/list.csv', index_col=0) 

# mark base sounds as original
sounds['origin'] = 'original'
sounds['shift'] = 0

new_sounds = pd.DataFrame({"file":files,
                      "note":notes,
                      "dynamics":dynamics,
                           "octave":octaves,
                      "pitch":pitches, 
                          'shift':shifts})
new_sounds['origin'] = 'transformed'
sounds = sounds.append(new_sounds)
sounds.to_csv('../experiment/sounds/list.csv')


scale by 0.97: Piano.mf.A1.t.shift-50.wav scale by 0.98: Piano.mf.A1.t.shift-40.wav scale by 0.98: Piano.mf.A1.t.shift-30.wav scale by 0.99: Piano.mf.A1.t.shift-20.wav scale by 0.99: Piano.mf.A1.t.shift-10.wav scale by 1.00: Piano.mf.A1.t.shift+0.wav scale by 1.01: Piano.mf.A1.t.shift+10.wav scale by 1.01: Piano.mf.A1.t.shift+20.wav scale by 1.02: Piano.mf.A1.t.shift+30.wav scale by 1.02: Piano.mf.A1.t.shift+40.wav scale by 0.97: Piano.mf.A2.t.shift-50.wav scale by 0.98: Piano.mf.A2.t.shift-40.wav scale by 0.98: Piano.mf.A2.t.shift-30.wav scale by 0.99: Piano.mf.A2.t.shift-20.wav scale by 0.99: Piano.mf.A2.t.shift-10.wav scale by 1.00: Piano.mf.A2.t.shift+0.wav scale by 1.01: Piano.mf.A2.t.shift+10.wav scale by 1.01: Piano.mf.A2.t.shift+20.wav scale by 1.02: Piano.mf.A2.t.shift+30.wav scale by 1.02: Piano.mf.A2.t.shift+40.wav scale by 0.97: Piano.mf.A3.t.shift-50.wav scale by 0.98: Piano.mf.A3.t.shift-40.wav scale by 0.98: Piano.mf.A3.t.shift-30.wav scale by 0.99: Piano.mf.A3.t.shift-2

scale by 0.98: Piano.mf.B7.t.shift-40.wav scale by 0.98: Piano.mf.B7.t.shift-30.wav scale by 0.99: Piano.mf.B7.t.shift-20.wav scale by 0.99: Piano.mf.B7.t.shift-10.wav scale by 1.00: Piano.mf.B7.t.shift+0.wav scale by 1.01: Piano.mf.B7.t.shift+10.wav scale by 1.01: Piano.mf.B7.t.shift+20.wav scale by 1.02: Piano.mf.B7.t.shift+30.wav scale by 1.02: Piano.mf.B7.t.shift+40.wav scale by 0.97: Piano.mf.Bb1.t.shift-50.wav scale by 0.98: Piano.mf.Bb1.t.shift-40.wav scale by 0.98: Piano.mf.Bb1.t.shift-30.wav scale by 0.99: Piano.mf.Bb1.t.shift-20.wav scale by 0.99: Piano.mf.Bb1.t.shift-10.wav scale by 1.00: Piano.mf.Bb1.t.shift+0.wav scale by 1.01: Piano.mf.Bb1.t.shift+10.wav scale by 1.01: Piano.mf.Bb1.t.shift+20.wav scale by 1.02: Piano.mf.Bb1.t.shift+30.wav scale by 1.02: Piano.mf.Bb1.t.shift+40.wav scale by 0.97: Piano.mf.Bb2.t.shift-50.wav scale by 0.98: Piano.mf.Bb2.t.shift-40.wav scale by 0.98: Piano.mf.Bb2.t.shift-30.wav scale by 0.99: Piano.mf.Bb2.t.shift-20.wav scale by 0.99: Piano.m

scale by 0.97: Piano.mf.D7.t.shift-50.wav scale by 0.98: Piano.mf.D7.t.shift-40.wav scale by 0.98: Piano.mf.D7.t.shift-30.wav scale by 0.99: Piano.mf.D7.t.shift-20.wav scale by 0.99: Piano.mf.D7.t.shift-10.wav scale by 1.00: Piano.mf.D7.t.shift+0.wav scale by 1.01: Piano.mf.D7.t.shift+10.wav scale by 1.01: Piano.mf.D7.t.shift+20.wav scale by 1.02: Piano.mf.D7.t.shift+30.wav scale by 1.02: Piano.mf.D7.t.shift+40.wav scale by 0.97: Piano.mf.Db1.t.shift-50.wav scale by 0.98: Piano.mf.Db1.t.shift-40.wav scale by 0.98: Piano.mf.Db1.t.shift-30.wav scale by 0.99: Piano.mf.Db1.t.shift-20.wav scale by 0.99: Piano.mf.Db1.t.shift-10.wav scale by 1.00: Piano.mf.Db1.t.shift+0.wav scale by 1.01: Piano.mf.Db1.t.shift+10.wav scale by 1.01: Piano.mf.Db1.t.shift+20.wav scale by 1.02: Piano.mf.Db1.t.shift+30.wav scale by 1.02: Piano.mf.Db1.t.shift+40.wav scale by 0.97: Piano.mf.Db2.t.shift-50.wav scale by 0.98: Piano.mf.Db2.t.shift-40.wav scale by 0.98: Piano.mf.Db2.t.shift-30.wav scale by 0.99: Piano.mf

scale by 0.97: Piano.mf.Eb7.t.shift-50.wav scale by 0.98: Piano.mf.Eb7.t.shift-40.wav scale by 0.98: Piano.mf.Eb7.t.shift-30.wav scale by 0.99: Piano.mf.Eb7.t.shift-20.wav scale by 0.99: Piano.mf.Eb7.t.shift-10.wav scale by 1.00: Piano.mf.Eb7.t.shift+0.wav scale by 1.01: Piano.mf.Eb7.t.shift+10.wav scale by 1.01: Piano.mf.Eb7.t.shift+20.wav scale by 1.02: Piano.mf.Eb7.t.shift+30.wav scale by 1.02: Piano.mf.Eb7.t.shift+40.wav scale by 0.97: Piano.mf.F1.t.shift-50.wav scale by 0.98: Piano.mf.F1.t.shift-40.wav scale by 0.98: Piano.mf.F1.t.shift-30.wav scale by 0.99: Piano.mf.F1.t.shift-20.wav scale by 0.99: Piano.mf.F1.t.shift-10.wav scale by 1.00: Piano.mf.F1.t.shift+0.wav scale by 1.01: Piano.mf.F1.t.shift+10.wav scale by 1.01: Piano.mf.F1.t.shift+20.wav scale by 1.02: Piano.mf.F1.t.shift+30.wav scale by 1.02: Piano.mf.F1.t.shift+40.wav scale by 0.97: Piano.mf.F2.t.shift-50.wav scale by 0.98: Piano.mf.F2.t.shift-40.wav scale by 0.98: Piano.mf.F2.t.shift-30.wav scale by 0.99: Piano.mf.F2

In [14]:
# convert back to pcm_s16le wav files
for input_file in glob.glob(sound_dir+"transformed/temp/*.wav"):
    output_file = sound_dir + "/transformed/" +os.path.basename(input_file)
    print(output_file+',', end='')
    convert_cmd = "ffmpeg -i " + input_file + " -y -acodec pcm_s16le -ar 44100 -ac 1 " + output_file
    os.system(convert_cmd)

../experiment/sounds//transformed/Piano.mf.A1.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.A1.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.A1.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.A1.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.A1.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.A1.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.A1.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.A1.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.A1.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.A1.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.A2.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.A2.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.A2.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.A2.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.A2.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.A2.t.shift-10.wav,../experiment/sounds//tran

../experiment/sounds//transformed/Piano.mf.Ab7.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.Ab7.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.Ab7.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.Ab7.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.Ab7.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.B0.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.B1.t.shift+0.wav,../experiment/sounds/

../experiment/sounds//transformed/Piano.mf.Bb6.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.Bb6.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.Bb6.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.Bb6.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.Bb6.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.Bb6.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.Bb6.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.Bb6.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.Bb6.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.Bb7.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.Bb7.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.Bb7.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.Bb7.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.Bb7.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.Bb7.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.Bb7.t.shift-20.wav,../experi

../experiment/sounds//transformed/Piano.mf.D4.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.D4.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.D4.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.D5.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.D6.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.D6.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.D6.t.shift+20.wav,../experiment/sounds//tran

../experiment/sounds//transformed/Piano.mf.E4.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.E4.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.E4.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.E4.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.E4.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.E4.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.E5.t.shift-50.wav,../experiment/sounds//tra

../experiment/sounds//transformed/Piano.mf.F4.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.F4.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.F4.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.F4.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.F4.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.F4.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.F4.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.F4.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.F4.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.F5.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.F5.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.F5.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.F5.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.F5.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.F5.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.F5.t.shift-20.wav,../experiment/sounds//tra

../experiment/sounds//transformed/Piano.mf.Gb3.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.Gb3.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.Gb3.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.Gb3.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift+10.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift+20.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift+30.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift+40.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift-10.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift-20.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift-30.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift-40.wav,../experiment/sounds//transformed/Piano.mf.Gb4.t.shift-50.wav,../experiment/sounds//transformed/Piano.mf.Gb5.t.shift+0.wav,../experiment/sounds//transformed/Piano.mf.Gb5.t.shift+10.wav,../experim

In [47]:
sounds.to_csv('../experiment/sounds/list.csv')